# Setup

In [88]:
# Imports
import xarray as xr
import numpy as np
import pandas as pd


from bokeh.io import show, output_notebook, reset_output
from bokeh.plotting import figure

#from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, RadioGroup, Select, Tabs, TableColumn, Panel
from bokeh.models import ColumnDataSource, CategoricalColorMapper, GroupFilter
from bokeh.transform import factor_cmap, factor_mark

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category10_10

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

# Create Dataset


In [89]:
ds = xr.open_dataset('waterhackweek_dalles_salmonfalls/waterhackweek_dalles_salmonfalls.nc')
ds

<xarray.Dataset>
Dimensions:           (downscale_method: 2, gcm: 10, outlets: 2, parameters: 4, rcp: 2, time: 54787)
Coordinates:
  * time              (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2099-12-31
  * outlets           (outlets) |S256 b'TDA' b'BRN'
  * parameters        (parameters) object 'PRMS' 'calib_inverse' 'ORNL' 'NCAR'
  * downscale_method  (downscale_method) object 'bcsd' 'maca'
  * gcm               (gcm) object 'hadgem2-cc' 'hadgem2-es' ... 'ccsm4'
  * rcp               (rcp) object 'rcp45' 'rcp85'
Data variables:
    streamflow        (time, outlets, parameters, downscale_method, gcm, rcp) float64 ...

In [90]:
# Ann. Peak Flows
df_max = ds.resample(time='Y').max().to_dataframe().reset_index()
df_max['metrics'] = 'Ann. Max Flow'
df_max['x'] = df_max['time'].dt.year
df_max['y'] = df_max['streamflow']
df_max.drop(['time', 'streamflow'], axis=1, inplace=True)

# Ann. Low Flows
df_min = ds.resample(time='Y').min().to_dataframe().reset_index()
df_min['metrics'] = 'Ann. Min Flow'
df_min['x'] = df_min['time'].dt.year
df_min['y'] = df_min['streamflow']
df_min.drop(['time', 'streamflow'], axis=1, inplace=True)

# Create full dataframe
df = pd.DataFrame()
df = df.append(df_max)
df = df.append(df_min)
df['outlets'] = df['outlets'].str.decode("utf-8")
df.head()


label_lookup = pd.DataFrame()
label_loopup = label_lookup.append(pd.DataFrame({"metric":['Ann. Max Flow'], 'x':['Year'], 'y':["Streamflow (CMS)"]}))
label_loopup = label_lookup.append(pd.DataFrame({"metric":['Ann. Min Flow'], 'x':['Year'], 'y':["Streamflow (CMS)"]}))




# Create plot

In [92]:
outlet_list = list(df.outlets.unique())
metrics_list = list(df.metrics.unique())
params=["Parameters", "RCP", "Downscale Method", "GCM"]
params_real = ["parameters", 'rcp', 'downscale_method', 'gcm']
gcm_list = list(df.gcm.unique())

In [122]:
def modify_doc(doc):
    def make_dataset(outlet_select, metric_select, group_select, gcm_selections):
        cf = df[(df['outlets'] == outlet_select) &
                (df['metrics'] == metric_select) & 
                (df['gcm'].isin(gcm_selections))]
        
        param = group_select                
        min_df = cf[['x', 'y', param]].groupby(['x', param]).min().reset_index()
        max_df = cf[['x', 'y', param]].groupby(['x', param]).max().reset_index()
        avg_df = cf[['x', 'y', param]].groupby(['x', param]).mean().reset_index()
                
        cf = pd.merge(avg_df, min_df, on=['x', param])
        cf = pd.merge(cf, max_df, on=['x', param])
        cf.columns = ['x', 'param', 'y_avg', 'y_min', 'y_max'] 
        cf['left'] = cf['x'] - 0.5        
        cf['right'] = cf['x'] + 0.5
        
        # Color each param differently
        cf['color'] = '#000000'
        for i, p in enumerate(df[param].unique()):
            cf.loc[cf['param']==p, 'color'] = Category10_10[i]
        return ColumnDataSource(cf)    
    
    
    def make_plot(src):
        p = figure(plot_width = 700, plot_height = 700, 
                  title = 'Title goes here',
                  x_axis_label = 'x-axis label goes here', y_axis_label = 'y-axis label goes here')
    
        p.quad(source=src, left='left', right='right', top='y_max', bottom='y_min', alpha=0.2, color='color')
        p.scatter(source=src, x='x', y='y_avg', legend='param', color='color')
        
        #p.multi_line(source=src, xs='x', ys='y_avg', color='color', legend='param')        
        #df = src.to_df()
        #for i in df.param.unique():
        #    i_df = df[df['param'] == i]
        #    i_color = i_df['color'].unique()[0]
        #    i_src = ColumnDataSource(i_df)
        #    p.line(source=i_src, x='x', y='y_avg', legend='param', color=i_color, line_width=2)
        
    
        return p
        

    def update(attr, old, new):
        outlet_to_plot = outlet_selector.value
        metric_to_plot = metric_selector.value
        group_selected = params_real[groupby_selector.active]
        gcms_to_plot = [gcm_selector.labels[i] for i in gcm_selector.active]
            
        new_src = make_dataset(outlet_to_plot, metric_to_plot, group_selected, gcms_to_plot)
        #new_line_src = make_lineset(src)
        src.data.update(new_src.data)
        #line_src.data.update(new_line_src)
        
        
    # Initialize widget controls
    outlet_selector = Select(title="Outlets", value = outlet_list[0], options=outlet_list)
    outlet_selector.on_change('value', update)

    groupby_selector = RadioGroup(labels=params, active=3)
    groupby_selector.on_change('active', update)

    gcm_selector = CheckboxGroup(labels=gcm_list, active=[0,1])
    gcm_selector.on_change('active', update)

    metric_selector = Select(title="Metrics", value=metrics_list[0], options=metrics_list)
    metric_selector.on_change('value', update)

    
    # Create initial dataset and plot
    src = make_dataset(outlet_selector.value, 
                       metric_selector.value, 
                       params_real[groupby_selector.active], 
                       [gcm_selector.labels[i] for i in gcm_selector.active])
    p = make_plot(src)
    
    
    # Put controls in a single element
    controls = WidgetBox(outlet_selector, metric_selector, groupby_selector, gcm_selector)
    
    # Create a row layout
    layout = row(p, controls)
    
    doc.add_root(layout)

    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [123]:

show(app)